<a href="https://colab.research.google.com/github/ilEnzio/DS-Unit-2-Linear-Models/blob/master/module4-logistic-regression/ERLE_GRANGER_DS18_LS_DS_214_assignment_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 4*

---

# Logistic Regression


## Assignment 🌯

You'll use a [**dataset of 400+ burrito reviews**](https://srcole.github.io/100burritos/). How accurately can you predict whether a burrito is rated 'Great'?

> We have developed a 10-dimensional system for rating the burritos in San Diego. ... Generate models for what makes a burrito great and investigate correlations in its dimensions.

- [ ] Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.
- [ ] Begin with baselines for classification.
- [ ] Use scikit-learn for logistic regression.
- [ ] Get your model's validation accuracy. (Multiple times if you try multiple iterations.)
- [ ] Get your model's test accuracy. (One time, at the end.)
- [ ] Commit your notebook to your fork of the GitHub repo.


## Stretch Goals

- [ ] Add your own stretch goal(s) !
- [ ] Make exploratory visualizations.
- [ ] Do one-hot encoding.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Get and plot your coefficients.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [2]:
# Load data downloaded from https://srcole.github.io/100burritos/
import pandas as pd
df = pd.read_csv(DATA_PATH+'burritos/burritos.csv')

In [3]:
# Derive binary classification target:
# We define a 'Great' burrito as having an
# overall rating of 4 or higher, on a 5 point scale.
# Drop unrated burritos.
df = df.dropna(subset=['overall'])
df['Great'] = df['overall'] >= 4

In [4]:
# Clean/combine the Burrito categories
df['Burrito'] = df['Burrito'].str.lower()

california = df['Burrito'].str.contains('california')
asada = df['Burrito'].str.contains('asada')
surf = df['Burrito'].str.contains('surf')
carnitas = df['Burrito'].str.contains('carnitas')

df.loc[california, 'Burrito'] = 'California'
df.loc[asada, 'Burrito'] = 'Asada'
df.loc[surf, 'Burrito'] = 'Surf & Turf'
df.loc[carnitas, 'Burrito'] = 'Carnitas'
df.loc[~california & ~asada & ~surf & ~carnitas, 'Burrito'] = 'Other'

In [5]:
# Drop some high cardinality categoricals
df = df.drop(columns=['Notes', 'Location', 'Reviewer', 'Address', 'URL', 'Neighborhood'])

In [6]:
# Drop some columns to prevent "leakage"
df = df.drop(columns=['Rec', 'overall'])

## Import Libraries and Modules

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


from sklearn.model_selection import train_test_split


# special sauce for custom method :) 
from collections import namedtuple

# ML models
from sklearn.linear_model import LinearRegression # old reliable model, for continous
from sklearn.linear_model import Ridge # RidgeRegression to guard against overfit
from sklearn.linear_model import LogisticRegression # used to predict categorical vectors


# Metric accessing functions
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Transformers
from category_encoders import OneHotEncoder # splits cat variable 
from sklearn.feature_selection import SelectKBest # selects highest correlating features
from sklearn.impute import SimpleImputer

# Lets us combine the steps for man
from sklearn.pipeline import Pipeline

## Some EDA / Wrangle

#### **Goal** - Since there are only **400 observations** (less for each subset!) I want to get the parameters down to 20 or less - Accounting for possible expansion due to encoding transformations.  

In [7]:
df.head()

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
0,California,1/18/2016,3.5,4.2,NaN,6.49,3.0,NaN,NaN,NaN,NaN,NaN,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,California,1/24/2016,3.5,3.3,NaN,5.45,3.5,NaN,NaN,NaN,NaN,NaN,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,Carnitas,1/24/2016,NaN,NaN,NaN,4.85,1.5,NaN,NaN,NaN,NaN,NaN,3.0,2.0,2.5,3.0,4.5,4.0,3.0,3.0,5.0,NaN,NaN,NaN,x,x,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,Asada,1/24/2016,NaN,NaN,NaN,5.25,2.0,NaN,NaN,NaN,NaN,NaN,3.0,2.0,3.5,3.0,4.0,5.0,4.0,4.0,5.0,NaN,NaN,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,California,1/27/2016,4.0,3.8,x,6.59,4.0,NaN,NaN,NaN,NaN,NaN,4.0,5.0,4.0,3.5,4.5,5.0,2.5,4.5,4.0,NaN,NaN,x,x,NaN,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [9]:
df.shape


(421, 59)

In [10]:
df.isnull().sum()

Burrito             0
Date                0
Yelp              334
Google            334
Chips             395
Cost                7
Hunger              3
Mass (g)          399
Density (g/mL)    399
Length            138
Circum            140
Volume            140
Tortilla            0
Temp               20
Meat               14
Fillings            3
Meat:filling        9
Uniformity          2
Salsa              25
Synergy             2
Wrap                3
Unreliable        388
NonSD             414
Beef              242
Pico              263
Guac              267
Cheese            262
Fries             294
Sour cream        329
Pork              370
Chicken           400
Shrimp            400
Fish              415
Rice              385
Beans             386
Lettuce           410
Tomato            414
Bell peper        414
Carrots           420
Cabbage           413
Sauce             383
Salsa.1           414
Cilantro          406
Onion             404
Taquito           417
Pineapple 

In [ ]:
df.describe()

In [30]:
df.describe(exclude="number")

,Burrito,Date,Chips,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
count,421,421,26,33,7,179,158,154,159,127,92,51,21,21,6,36,35,11,7,7,1,8,38,7,15,17,4,7,2,4,4,1,5,3,3,2,13,3,1,421
unique,5,169,4,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2,1,1,1,1,1,1,1,1,1,2,1,2
top,California,8/30/2016,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,False
freq,169,29,21,33,5,137,127,114,128,102,67,36,20,17,4,26,27,9,5,4,1,6,33,6,9,9,3,5,2,4,4,1,5,3,3,2,13,2,1,239


In [11]:
df.columns

Index(['Burrito', 'Date', 'Yelp', 'Google', 'Chips', 'Cost', 'Hunger',
       'Mass (g)', 'Density (g/mL)', 'Length', 'Circum', 'Volume', 'Tortilla',
       'Temp', 'Meat', 'Fillings', 'Meat:filling', 'Uniformity', 'Salsa',
       'Synergy', 'Wrap', 'Unreliable', 'NonSD', 'Beef', 'Pico', 'Guac',
       'Cheese', 'Fries', 'Sour cream', 'Pork', 'Chicken', 'Shrimp', 'Fish',
       'Rice', 'Beans', 'Lettuce', 'Tomato', 'Bell peper', 'Carrots',
       'Cabbage', 'Sauce', 'Salsa.1', 'Cilantro', 'Onion', 'Taquito',
       'Pineapple', 'Ham', 'Chile relleno', 'Nopales', 'Lobster', 'Queso',
       'Egg', 'Mushroom', 'Bacon', 'Sushi', 'Avocado', 'Corn', 'Zucchini',
       'Great'],
      dtype='object')

In [37]:
df.shape

(421, 59)

In [33]:
df["Guac"].value_counts()

x    114
X     40
Name: Guac, dtype: int64

In [32]:
df['Pico'].value_counts()

x    127
X     31
Name: Pico, dtype: int64

In [31]:
df['Chips'].value_counts()

x      21
X       3
No      1
Yes     1
Name: Chips, dtype: int64

In [29]:
df['Pork'].value_counts()

x    36
X    15
Name: Pork, dtype: int64

In [22]:
df['Beef'].value_counts()

x    137
X     42
Name: Beef, dtype: int64

In [ ]:
df['Beef'].value_counts()

In [16]:
df["Unreliable"].value_counts()

x    33
Name: Unreliable, dtype: int64

In [23]:
df['Tortilla'].nunique()

18

In [24]:
df['Tortilla'].value_counts()

4.00    130
3.00    100
3.50     63
4.50     30
2.00     27
5.00     25
2.50     20
3.80      7
1.50      6
4.20      2
3.75      2
3.20      2
3.60      2
2.10      1
4.80      1
1.00      1
1.40      1
2.80      1
Name: Tortilla, dtype: int64

In [20]:
df["Meat"].value_counts()

4.00    120
3.50     69
3.00     62
4.50     37
5.00     35
2.00     25
2.50     18
1.50      5
3.80      5
4.20      4
3.30      4
4.30      3
1.00      3
3.20      3
3.70      3
2.60      2
3.75      2
2.75      2
4.70      1
2.70      1
2.80      1
4.10      1
3.90      1
Name: Meat, dtype: int64

In [19]:
df['Length'].value_counts()

20.00    34
19.00    29
18.00    24
18.50    23
19.50    21
21.00    19
20.50    17
22.50    16
23.00    16
22.00    16
17.00    14
21.50    13
17.50     9
16.50     5
23.50     5
24.00     4
25.00     2
25.50     2
15.00     2
15.50     2
24.50     2
21.30     1
20.75     1
16.00     1
26.00     1
21.59     1
17.78     1
17.70     1
22.20     1
Name: Length, dtype: int64

In [12]:
# Drop Mass, Density, Google, Yelp, Date, 'Unreliable', Circum, Volume, Queso
# Temp, 
# ingredients I might do 
# Fill with mean: length
# Engineer: 
# Tortilla, Meat, Synergy, Wrap, change to category; 
# Beef, Pico... - replace "X" or"x" with 1,
# 
# my strategy basically involves looking as the standard dev
# so... Keep:  Length, Meat, Tortilla, Synergy, Wrap, Pico, Beef,  'Guac',
#  'Cheese', 'Fries', 'Sour cream', 'Pork', 'Chicken', 'Shrimp', 'Fish',
#  'Rice', 'Beans', 'Lettuce', 'Tomato', 'Bell peper', 'Carrots',
#  'Cabbage', 'Sauce', 'Salsa.1', 'Cilantro', 'Onion', 'Taquito',
#  'Pineapple', 'Ham', 'Chile relleno', 'Nopales', 'Lobster', 'Queso',
#  'Egg', 'Mushroom', 'Bacon', 'Sushi', 'Avocado', 'Corn', 'Zucchini',
# 

In [50]:
data = ['Burrito', 'Yelp', 'Google', 'Chips', 'Cost', 'Hunger',
       'Mass (g)', 'Density (g/mL)','Circum', 'Volume','Temp', 'Fillings', 
      'Meat:filling', 'Uniformity', 'Unreliable', 'NonSD']

In [52]:
burr_ingredients_df = df.drop(labels=data, axis=1)
burr_ingredients_df.shape

(421, 43)

In [53]:
burr_ingredients_df.columns

Index(['Date', 'Length', 'Tortilla', 'Meat', 'Salsa', 'Synergy', 'Wrap',
       'Beef', 'Pico', 'Guac', 'Cheese', 'Fries', 'Sour cream', 'Pork',
       'Chicken', 'Shrimp', 'Fish', 'Rice', 'Beans', 'Lettuce', 'Tomato',
       'Bell peper', 'Carrots', 'Cabbage', 'Sauce', 'Salsa.1', 'Cilantro',
       'Onion', 'Taquito', 'Pineapple', 'Ham', 'Chile relleno', 'Nopales',
       'Lobster', 'Queso', 'Egg', 'Mushroom', 'Bacon', 'Sushi', 'Avocado',
       'Corn', 'Zucchini', 'Great'],
      dtype='object')

In [ ]:
burr_ingredients_df.info()

In [54]:
new_types = {'Tortilla' : 'category', 'Meat' : 'category',
             'Synergy' : 'category', 'Wrap' : 'category',
             'Date':"datetime64"}

In [55]:
burr_ingredients_df= burr_ingredients_df.astype(new_types)
burr_ingredients_df.columns

Index(['Date', 'Length', 'Tortilla', 'Meat', 'Salsa', 'Synergy', 'Wrap',
       'Beef', 'Pico', 'Guac', 'Cheese', 'Fries', 'Sour cream', 'Pork',
       'Chicken', 'Shrimp', 'Fish', 'Rice', 'Beans', 'Lettuce', 'Tomato',
       'Bell peper', 'Carrots', 'Cabbage', 'Sauce', 'Salsa.1', 'Cilantro',
       'Onion', 'Taquito', 'Pineapple', 'Ham', 'Chile relleno', 'Nopales',
       'Lobster', 'Queso', 'Egg', 'Mushroom', 'Bacon', 'Sushi', 'Avocado',
       'Corn', 'Zucchini', 'Great'],
      dtype='object')

In [60]:
burr_ingredients_df.shape

(421, 43)

## Because we're going to use imputer I think we are good to go! Create the Feature Matrix.  

In [58]:
data_target = {"Great" : burr_ingredients_df["Great"]}

In [59]:
X_matrix = burr_ingredients_df.drop(labels="Great", axis=1)
X_matrix.shape

(421, 42)

In [61]:
y_target = pd.DataFrame(data_target)
y_target.shape

(421, 1)

# Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.

In [62]:
X_matrix["Date"].dt.year.value_counts()

2016    296
2017     85
2018     27
2019     10
2026      1
2015      1
2011      1
Name: Date, dtype: int64

In [68]:
cond_2017_Temptrain = X_matrix["Date"].dt.year <= 2017
cond_2017_Temptrain

0       True
1       True
2       True
3       True
4       True
       ...  
418    False
419    False
420    False
421    False
422    False
Name: Date, Length: 421, dtype: bool

0      False
1      False
2      False
3      False
4      False
       ...  
418     True
419     True
420     True
421     True
422     True
Name: Date, Length: 421, dtype: bool

In [70]:
X_train, y_train = X_matrix.loc[cond_2017_Temptrain], y_target.loc[cond_2017_Temptrain]
X_test, y_test = X_matrix.loc[~cond_2017_Temptrain], y_target.loc[~cond_2017_Temptrain]

In [71]:
X_test.shape

(38, 42)

In [ ]:
cond_2016_train = X_matrix["Date"].dt.year >= 2017
cond_2016_train

In [ ]:
cond_2017_val = X_matrix["Date"].dt.year == 2017
cond_2017_val

In [75]:
X_train, y_train = X_matrix.loc[cond_2016_train], y_target.loc[cond_2016_train]
X_val, y_val = X_matrix.loc[cond_2017_val], y_target.loc[cond_2017_val]


In [77]:
print(X_matrix.shape)
print()
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(421, 42)

(298, 42)
(85, 42)
(38, 42)


In [78]:
print(y_target.shape)
print()
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(421, 1)

(298, 1)
(85, 1)
(38, 1)


In [80]:
y_train.head()

,Great
0,False
1,False
2,False
3,False
4,True


In [84]:
y_train['Great'].value_counts(normalize=True).max()

0.5906040268456376

## Establish a Baseline 
#### Since the **Target Vector** is Categorical use a
####** majority classifier**. 

In [85]:
print(f"Our Basline Accuracy Score will be: {y_train['Great'].value_counts(normalize=True).max()}")

Our Basline Accuracy Score will be: 0.5906040268456376


### Build Model
#### Use a pipeline: Imputer, Logistic Regression

In [ ]:
log_reg_model = Pipeline()